In [1]:
# Setup and Imports
import pandas as pd
import requests
import h3
import time
from typing import Dict, Tuple
from datetime import datetime

In [3]:
# Configuration
API_KEY = ""  # Replace with your Google Maps API key
INPUT_FILE = "Output/Updated_RDO No. 49 - North Makati City.xlsx"  # Replace with your file name
H3_RESOLUTION = 9  # Adjust resolution as needed (0-15)

In [4]:
# Read the Excel file
df = pd.read_excel(INPUT_FILE)
print("Data loaded successfully!")
print(f"Number of addresses to process: {len(df)}")
df.head()

Data loaded successfully!
Number of addresses to process: 939


,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM
0,NCR,MAKATI CITY,BEL-AIR,AMAPOLA,Estrella - Mercedes,RR,250000.0
1,NCR,MAKATI CITY,BEL-AIR,ANTARES,Jupiter - Mercedes,RR,250000.0
2,NCR,MAKATI CITY,BEL-AIR,ANZA,Makati Ave. - Polaris,CR,350000.0
3,NCR,MAKATI CITY,BEL-AIR,AQUARIUS,Hydra - Solar,RR,250000.0
4,NCR,MAKATI CITY,BEL-AIR,ARIES,Mercedes - Taurus,RR,250000.0


In [6]:
small_df = df.sample(n=3).copy()
small_df

,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM
453,NCR,MAKATI CITY,POBLACION,AMORSOLO SQUARE,"Amorsolo Drive, Rockwell Center",CC,300000.0
603,NCR,MAKATI CITY,SAN ANTONIO,ANUBBING,Mayapis - Kamagong,CR,100000.0
155,NCR,MAKATI CITY,GUADALUPE VIEJO,1 PROSCENIUM,"The Proscenium, Rockwell, Dr Jose P. Rizal Ave",CC,300000.0


In [9]:
# Function to format address
def format_address(row: pd.Series) -> str:
    """Format address components into a single string"""
    components = []
    if pd.notna(row.get('Street/Subdivision')):
        components.append(str(row['Street/Subdivision']))
    if pd.notna(row.get('Barangay')):
        components.append(f"{row['Barangay']}")
    if pd.notna(row.get('City/Municipality')):
        components.append(str(row['City/Municipality']))
    if pd.notna(row.get('Province')):
        components.append(str(row['Province']))
    components.append('Philippines')
    return ", ".join(components)

# Test address formatting with first row
test_address = format_address(df.iloc[0])
print("Sample formatted address:", test_address)

Sample formatted address: AMAPOLA, BEL-AIR, MAKATI CITY, NCR, Philippines


In [ ]:
# Function to geocode address
def geocode_address(address: str, api_key: str) -> Tuple[float, float, Dict]:
    """Geocode address using Google's Geocoding API"""
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': address,
        'key': api_key,
        'region': 'ph'
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if data['status'] == 'OK':
        location = data['results'][0]['geometry']['location']
        return location['lat'], location['lng'], data['results'][0]
    else:
        raise Exception(f"Geocoding failed: {data['status']}")

# Test geocoding with sample address
try:
    lat, lng, data = geocode_address(test_address, API_KEY)
    print(f"Test geocoding successful!")
    print(f"Latitude: {lat}")
    print(f"Longitude: {lng}")
    print(f"Formatted Address: {data['formatted_address']}")
except Exception as e:
    print(f"Test geocoding failed: {str(e)}")

In [12]:
# Function to get H3 index
def get_h3_index(lat: float, lng: float, resolution: int = 9) -> str:
    """Convert coordinates to H3 index"""
    return h3.latlng_to_cell(lat, lng, resolution)

In [18]:
# Process all addresses
results = []

for idx, row in df.iterrows():
    try:
        address = format_address(row)
        print(f"Processing {idx + 1}/{len(df)}: {address}")
        
        # Add delay to respect API rate limits
        time.sleep(0.1)
        
        lat, lng, google_data = geocode_address(address, API_KEY)
        h3_index = get_h3_index(lat, lng, H3_RESOLUTION)
        
        results.append({
            'original_address': address,
            'latitude': lat,
            'longitude': lng,
            'h3_index': h3_index,
            'google_formatted_address': google_data['formatted_address'],
            'google_place_id': google_data['place_id'],
            'google_location_type': google_data['geometry']['location_type']
        })
        
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        results.append({
            'original_address': address,
            'error': str(e)
        })

Processing 1/939: AMAPOLA, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 2/939: ANTARES, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 3/939: ANZA, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 4/939: AQUARIUS, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 5/939: ARIES, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 6/939: ASTEROID, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 7/939: ASTRA I, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 8/939: AYALA AVENUE (EXTENSION)***, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 9/939: CANOPUS, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 10/939: COMET, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 11/939: CONSTELLATION, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 12/939: ESTRELLA, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 13/939: GALAXY, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 14/939: HERCULES, BEL-AIR, MAKATI CITY, NCR, Philippines
Processing 15/939: HYDRA, BEL-AIR, MAKATI CITY, N

In [19]:
# Convert results to DataFrame and merge with original data
results_df = pd.DataFrame(results)
output_df = pd.concat([df, results_df], axis=1)

In [20]:
# Display sample results
print("\nSample of processed data:")
output_df.head(5)


Sample of processed data:


,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM,original_address,latitude,longitude,h3_index,google_formatted_address,google_place_id,google_location_type
0,NCR,MAKATI CITY,BEL-AIR,AMAPOLA,Estrella - Mercedes,RR,250000.0,"AMAPOLA, BEL-AIR, MAKATI CITY, NCR, Philippines",14.562833,121.037264,89694ec0e4bffff,"Amapola, Makati, Kalakhang Maynila, Philippines",Ei9BbWFwb2xhLCBNYWthdGksIEthbGFraGFuZyBNYXluaW...,GEOMETRIC_CENTER
1,NCR,MAKATI CITY,BEL-AIR,ANTARES,Jupiter - Mercedes,RR,250000.0,"ANTARES, BEL-AIR, MAKATI CITY, NCR, Philippines",14.558207,121.034377,89694ec1d8fffff,"Antares, Makati, Metro Manila, Philippines",ChIJ-c7pNv7IlzMR0gWibCBQkZs,GEOMETRIC_CENTER
2,NCR,MAKATI CITY,BEL-AIR,ANZA,Makati Ave. - Polaris,CR,350000.0,"ANZA, BEL-AIR, MAKATI CITY, NCR, Philippines",14.562393,121.028610,89694ec032bffff,"Anza, Makati, Metro Manila, Philippines",ChIJCeHlCKrJlzMRYt2_0QtgBFY,GEOMETRIC_CENTER
3,NCR,MAKATI CITY,BEL-AIR,AQUARIUS,Hydra - Solar,RR,250000.0,"AQUARIUS, BEL-AIR, MAKATI CITY, NCR, Philippines",14.561553,121.036326,89694ec0e4bffff,"Aquarius, Makati, Metro Manila, Philippines",EitBcXVhcml1cywgTWFrYXRpLCBNZXRybyBNYW5pbGEsIF...,GEOMETRIC_CENTER
4,NCR,MAKATI CITY,BEL-AIR,ARIES,Mercedes - Taurus,RR,250000.0,"ARIES, BEL-AIR, MAKATI CITY, NCR, Philippines",14.562170,121.036327,89694ec0e4bffff,"Aries, Makati, Metro Manila, Philippines",ChIJ-XnUe1XIlzMRYoMIuFoH5m0,GEOMETRIC_CENTER


In [ ]:
# Save results to Excel
output_file = f"geocoded_addresses_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
output_df.to_excel(output_file, index=False)
print(f"\nProcessing complete. Results saved to {output_file}")

In [21]:
# Optional: Basic analysis of results
print("\nProcessing Summary:")
print(f"Total addresses processed: {len(output_df)}")
print(f"Successful geocoding: {output_df['latitude'].notna().sum()}")
print(f"Failed geocoding: {output_df['latitude'].isna().sum()}")


Processing Summary:
Total addresses processed: 939
Successful geocoding: 939
Failed geocoding: 0


In [22]:
# Optional: Display unique location types
if 'google_location_type' in output_df.columns:
    print("\nLocation type distribution:")
    print(output_df['google_location_type'].value_counts())


Location type distribution:
google_location_type
GEOMETRIC_CENTER      640
ROOFTOP               220
APPROXIMATE            76
RANGE_INTERPOLATED      3
Name: count, dtype: int64


# Geocoding Viz

In [35]:
import pandas as pd
import folium
import h3
import json
import numpy as np
from branca.colormap import LinearColormap

def h3_hexagon_to_coordinates(h3_address):
    """Convert H3 address to coordinates for plotting"""
    boundary = h3.cell_to_boundary(h3_address)
    return [[coord[0], coord[1]] for coord in boundary]

# Read and clean the data
df = output_df

# Clean and convert ZV/SQM to numeric, handling any non-numeric values
df['ZV/SQM'] = pd.to_numeric(df['ZV/SQM'], errors='coerce')

# Remove any rows where ZV/SQM is NaN
df = df.dropna(subset=['ZV/SQM'])

# Sort the values to ensure proper color mapping
df = df.sort_values('ZV/SQM')

# Create a base map centered on Makati
center_lat = df['latitude'].mean()
center_lng = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lng], zoom_start=14)

# Create color map based on quantiles for better distribution
q_25 = df['ZV/SQM'].quantile(0.25)
q_50 = df['ZV/SQM'].quantile(0.50)
q_75 = df['ZV/SQM'].quantile(0.75)
max_value = df['ZV/SQM'].max()

colormap = LinearColormap(
    colors=['#fee5d9', '#fcae91', '#fb6a4a', '#cb181d'],
    vmin=df['ZV/SQM'].min(),
    vmax=max_value,
    caption='Zone Value per Square Meter (PHP)'
)

# Create a dictionary to store hexagons by H3 index to avoid duplicates
hexagon_dict = {}

# Process each row
for idx, row in df.iterrows():
    try:
        # Get hexagon coordinates
        hex_coordinates = h3_hexagon_to_coordinates(row['h3_index'])
        
        # Store the highest ZV/SQM value for each hexagon
        # TODO: ask if highest or average
        if row['h3_index'] not in hexagon_dict or hexagon_dict[row['h3_index']]['value'] < row['ZV/SQM']:
            hexagon_dict[row['h3_index']] = {
                'coordinates': hex_coordinates,
                'value': row['ZV/SQM'],
                'popup_text': f"""
                Address: {row['google_formatted_address']}<br>
                Zone Value: ₱{row['ZV/SQM']:,.2f}/sqm<br>
                """
            }
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")

# Add hexagons to the map
for h3_index, data in hexagon_dict.items():
    try:
        # Calculate color based on ZV/SQM
        norm_value = (data['value'] - df['ZV/SQM'].min()) / (max_value - df['ZV/SQM'].min())
        color = colormap(data['value'])
        
        # Add polygon to map
        folium.Polygon(
            locations=data['coordinates'],
            color='black',
            weight=1,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=data['popup_text']
        ).add_to(m)
    except Exception as e:
        print(f"Error adding hexagon {h3_index}: {str(e)}")

# Add the colormap to the map
colormap.add_to(m)

# Save the map
m.save('makati_zoning_map.html')

# Print statistics
print("\nZone Value Statistics:")
print(df['ZV/SQM'].describe())

print("\nValue Ranges:")
print(f"0-25th percentile: ₱{q_25:,.2f}/sqm")
print(f"25-50th percentile: ₱{q_50:,.2f}/sqm")
print(f"50-75th percentile: ₱{q_75:,.2f}/sqm")
print(f"75-100th percentile: ₱{max_value:,.2f}/sqm")

print("\nClassification Distribution:")
print(df['Classification'].value_counts())


Zone Value Statistics:
count       926.000000
mean     152545.896328
std       73678.209580
min       38500.000000
25%      100000.000000
50%      130000.000000
75%      180000.000000
max      500000.000000
Name: ZV/SQM, dtype: float64

Value Ranges:
0-25th percentile: ₱100,000.00/sqm
25-50th percentile: ₱130,000.00/sqm
50-75th percentile: ₱180,000.00/sqm
75-100th percentile: ₱500,000.00/sqm

Classification Distribution:
Classification
RR    366
CR    177
PS    127
RC    120
CC    115
X      19
I       2
Name: count, dtype: int64


In [36]:
display(m)

In [24]:
def h3_hexagon_to_coordinates(h3_address):
    """Convert H3 address to coordinates for plotting"""
    boundary = h3.cell_to_boundary(h3_address)
    # Folium expects coordinates in lat,lng format
    return [[coord[0], coord[1]] for coord in boundary]

In [28]:
output_df['ZV/SQM'] = pd.to_numeric(output_df['ZV/SQM'], errors='coerce')

In [30]:
# Create a base map centered on Makati
center_lat = output_df['latitude'].mean()
center_lng = output_df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lng], zoom_start=14)

In [31]:
# Create color map based on ZV/SQM
min_value = output_df['ZV/SQM'].min()
max_value = output_df['ZV/SQM'].max()
colormap = LinearColormap(
    colors=['yellow', 'orange', 'red'],
    vmin=min_value,
    vmax=max_value,
    caption='Zone Value per Square Meter (PHP)'
)

In [32]:
# Add hexagons to the map
for idx, row in output_df.iterrows():
    # Get hexagon coordinates
    hex_coordinates = h3_hexagon_to_coordinates(row['h3_index'])
    
    # Calculate color based on ZV/SQM
    color = colormap(row['ZV/SQM'])
    
    # Create popup text
    popup_text = f"""
    Address: {row['google_formatted_address']}<br>
    Zone Value: ₱{row['ZV/SQM']:,.2f}/sqm<br>
    Classification: {row['Classification']}<br>
    Barangay: {row['Barangay']}<br>
    """
    
    # Add polygon to map
    folium.Polygon(
        locations=hex_coordinates,
        color='black',
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(m)

ValueError: Thresholds are not sorted.

In [ ]:
# Add the colormap to the map
colormap.add_to(m)

In [ ]:
# Save the map
m.save('makati_zoning_map.html')

# Optional: Display some statistics about the zones
print("\nZone Value Statistics:")
print(output_df['ZV/SQM'].describe())

print("\nClassification Distribution:")
print(output_df['Classification'].value_counts())